# Формулировка задания

Используя Python и модуль requests и bs4 написать скрипт, извлекающий новости (отдельно заголовок, анотацию, авторов) из веб-страницы новостного агенства. Требуется написать такой скрипт, который будучи запущен на сутки автоматически выделит и скачает все статьи, которые будут опубликованы за этот период (т.е. новые) и будут содержать упоминания Республиканской и Демократической партий США (выбираете любую интересную тему). Новостной сайт должен быть уникален и не повторяться у других студентов (в группе).

In [ ]:
from bs4 import BeautifulSoup
import	requests
import re
import time

In [ ]:
url = 'https://24.kg'

In [ ]:
themes = ['Нарын','Бишкек','РФ','Эрдоган']

In [ ]:
path = '/content/drive/MyDrive/STUDY/Скриптовые языки/articles/'

In [ ]:
#получаем автора, заголовок и текст статьи по url
def getArticle(url):
  article = requests.get(url)
  soup = BeautifulSoup(article.text, 'html.parser')
  author = soup.find("span", {"itemprop": "author"})
  if author is not None: 
    author = author.find("span", {"itemprop": "name"})
    authorName = author.text
  else: authorName = ''

  title = soup.find("h1", {"class": "newsTitle"})
    
  body = soup.find("div", {"itemprop": "articleBody"}).find_all("p")


  articleText = ''
  for data in body:
    articleText += data.text

  return {'url': url, 'author': authorName, 'title': title.text, 'text': articleText}

In [ ]:
#проверка упоминания в статье
def checkTheme(article, themes):
  for t in themes:  
    #index = article['title'].find(t)
    m_t = re.search(t, article['title'], re.IGNORECASE)
    m_b = re.search(t, article['text'], re.IGNORECASE)
    if m_t is not None:
      #print(m_t.group(0))
      return True
    elif m_b is not None:
      #print(m_b.group(0))
      return True
  return False

In [ ]:
#сохраняет разметку
def saveAsHtml(url, path, name):
  page = requests.get(url)
  with open(path + name + '.html', "w") as f:
    f.write(page.text)

In [ ]:
#пометка в бэклоге
def writeToBacklog(article, path, name):

  t = time.strftime("%D %H:%M:%S", time.localtime(time.time() + 7*60*60))
  with open(path + name, "a") as f:
    f.write(str(t) + '\n')
    f.write(article['url'] + '\n')
    f.write(article['author'] + '\n')
    f.write(article['title'] + '\n')
    f.write(article['text'] + '\n')
    f.write('\n')

In [ ]:
#считывание ссылок новостей с главной страницы
def getNewsLinks(url):
  site = requests.get(url)
  soup = BeautifulSoup(site.text, 'html.parser')

  news = soup.find("div", {"class": "lineNews"}).find_all("div", {"class": "one"})
  newsLinks = []

  for data in news:
    a = data.find('a')
    if a is not None:
        newsLinks.append(url + a['href'])

  return newsLinks

In [ ]:
#выявление вновь появившихся новостей, которые нужно проверить на тему
def getNewArticles(oldList, newList):
  newArticles = []

  for url in newList:
    if not(url in oldList):
      newArticles.append(url)

  return newArticles


In [ ]:
#новые статьи с заданной тематикой
def getArticlesWithThemes(shouldFiltered, themes):
  articlesWithThemes = []

  for link in shouldFiltered:
    article = getArticle(link)
    if checkTheme(article, themes):
      articlesWithThemes.append(article)

  return articlesWithThemes

In [ ]:
max_time = int(60*60*6) # 60*60*3 секунды
start_time = time.time()  # время начала работы

filteredArticles = []
newsLinks = getNewsLinks(url) #нужны новые

while (time.time() - start_time) < max_time:
    
    #print(time.time())
    print(time.strftime("%D %H:%M:%S", time.localtime(time.time() + 7*60*60)))

    newArticlesLinks = getNewsLinks(url)
    shouldFiltered = getNewArticles(newsLinks, newArticlesLinks)
    
    if (len(shouldFiltered)):
      articlesWithThemes = getArticlesWithThemes(shouldFiltered, themes)
      for i in range(0, len(articlesWithThemes)):
        saveAsHtml(articlesWithThemes[i]['url'], path, str(len(filteredArticles) + i))
        writeToBacklog(articlesWithThemes[i], path, 'Backlog.docx')

      filteredArticles.extend(articlesWithThemes)

    newsLinks = newArticlesLinks

    time.sleep(60*20) #60*20

05/29/23 11:00:14
05/29/23 11:20:16
05/29/23 11:40:18
05/29/23 12:00:24
05/29/23 12:20:27
05/29/23 12:40:36
05/29/23 13:00:41
05/29/23 13:20:47
05/29/23 13:40:56
05/29/23 14:01:02
05/29/23 14:21:08
05/29/23 14:41:09
05/29/23 15:01:14
05/29/23 15:21:18
05/29/23 15:41:21
05/29/23 16:01:25
05/29/23 16:21:29
05/29/23 16:41:32


In [ ]:
newArticlesLinks

['https://24.kg/biznes_info/266712_uspevayte_uvas_tri_dnya_chtobyi_podklyuchit_yandeks_besplatno_na90dney/',
 'https://24.kg/vlast/266709_deputata_ulana_primova_kritikuyut_kollegi_komitet_nevasha_sobstvennost/',
 'https://24.kg/vlast/266705_eto_nezakonno_adahan_madumarov_obvinyaet_ulana_primova_vnarushenii_reglamenta_/',
 'https://24.kg/vlast/266704_vbros_iprovokatsiya_damira_niyazalieva_otvetila_naobvineniya_vpodkupe/',
 'https://24.kg/obschestvo/266707_feyerverk_vshkole_i_o_direktora_izavuch_uvolenyi/',
 'https://24.kg/vlast/266701_den_eaes_sadyir_japarov_pozdravil_partnerov_posoyuzu/',
 'https://24.kg/obschestvo/266700_ukaz_prezidenta_vignor_roditeli_uchenikov_shkolyi_38bishkeka_jaluyutsya_napoboryi/',
 'https://24.kg/vlast/266697_vkyirgyizstane_prosyat_stroit_novyie_sportobyektyi_suchetom_potrebnostey_lovz/',
 'https://24.kg/vlast/266698_vleylekskom_rayone_otkryili_pogranzastavyi_ipostyi/',
 'https://24.kg/vlast/266692_vkyirgyizstane_predlagayut_otmenit_litsenzirovanie_detsadov_iat

In [ ]:
shouldFiltered

[]

In [ ]:
articlesWithThemes

[]

In [ ]:
filteredArticles

[{'url': 'https://24.kg/obschestvo/266664_trotuaryi_parkovki_ostanovka_chto_hotyat_sdelat_vozle_alamedinskogo_ryinka_/',
  'author': 'Мария ОРЛОВА',
  'title': 'Тротуары, парковки, остановка. Что хотят сделать возле Аламединского рынка ',
  'text': 'Глава Свердловского района Бишкека Максатбек Саргазаков совместно с\xa0представителями отдела дорожного надзора УПСМ ГУВД, МП\xa0«Бишкекасфальтсервис» и\xa0департамента транспорта и\xa0развития дорожно-транспортной инфраструктуры провел обход. Об\xa0этом сообщила пресс-служба мэрии.По\xa0ее\xa0данным, инспекция прошла по\xa0улице Курманжан Датки от\xa0проспекта Чуй до\xa0проспекта Жибек Жолу, а\xa0также по\xa0улице Буденного.МП\xa0«Бишкекасфальтсервис» поручено провести сметные расчеты на\xa0капитальный ремонт тротуара с\xa0обеих сторон улицы Курманжан Датки, а\xa0отделу экономики и\xa0финансов\xa0— инвентаризацию жилых помещений и\xa0хозяйствующих субъектов, препятствующих строительству тротуара.Кроме того, проверили санитарное состояние с